To compare some of the aspects (ncells, density, depth, etc) between the $\Delta^{++} \longrightarrow \pi^+$ and pure $\pi^+$ sample 

In [5]:
#imports
import uproot as ur
import awkward as ak
import numpy as np
import pandas as pd

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "6" #specify GPU
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import graph_util_splitdelta as gu
import plot_util as pu

from scipy.interpolate import interp1d
import scipy.stats as stats

import Week7_PFNutils as W

import convert_sets_splitdelta as cs


In [2]:
# metadata
layers = ["EMB1", "EMB2", "EMB3", "TileBar0", "TileBar1", "TileBar2"]#calorimeter layer names, B=barrel, EM = for EM showers, Tile=hadronic
cell_size_phi = [0.098, 0.0245, 0.0245, 0.1, 0.1, 0.1]#cell dimensions in these units
cell_size_eta = [0.0031, 0.025, 0.05, 0.1, 0.1, 0.2]
len_phi = [4, 16, 16, 4, 4, 4] #'image sizes' in phi/eta coords
len_eta = [128, 16, 8, 4, 4, 2]

In [3]:
import atlas_mpl_style as ampl #makes matplotlib follow atlas formal style
ampl.use_atlas_style()

In [10]:
#import all the pi+ data
path = '/fast_scratch/atlas_images/v7/'

branchpip = ['clusterIndex', 'truthE', 'nCluster', 'clusterE', 'clusterECalib','clusterPt', 'clusterEta', 
            'clusterPhi', 'cluster_nCells', 'cluster_sumCellE', 'cluster_ENG_CALIB_TOT', 'cluster_ENG_CALIB_OUT_T',
            'cluster_ENG_CALIB_DEAD_TOT', 'cluster_EM_PROBABILITY', 'cluster_HAD_WEIGHT', 'cluster_CENTER_MAG', 
            'cluster_FIRST_ENG_DENS', 'cluster_cellE_norm']
branchdel = ['cluster_hitsTruthE', 'cluster_hitsTruthIndex', 'nCluster', 'truthPartPdgId', 'cluster_nCells',
             'truthPartEta', 'truthPartPhi', 
         'truthPartPt', 'truthPartMass', 'cluster_E', 'cluster_Eta', 'cluster_Phi','cluster_Pt']

pp_file = ur.open(path+'piplus.root') 
pp_tree = pp_file['ClusterTree']
pp = pd.DataFrame(pp_tree.arrays(expressions = branchpip, library='np')) 

In [11]:
model_path = '/fast_scratch/atlas_images/v01-45/' 
events = ur.open(model_path+'delta_medium.root')
tree = events['EventTree']

delbranches = tree.arrays(expressions=branchdel)

KeyboardInterrupt: 